<a href="https://colab.research.google.com/github/mahendra204/Machine-Learning/blob/main/melbourne_snapshot_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'melbourne-housing-snapshot:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2709%2F38454%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240207%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240207T114957Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D910e3a99b2b008040aab95f6c101b8386e611e7a96dc300b046df403f9b21acd5ed9e961283f252df58afc8151633d371dfb1376cc9b7907a53fdacb43dd974608e3eb963292faae9d8699d70169096349156dd0e12c5b8de4ebb04a4153368effd2607b7338a876f475999221974f0f859e43c0c29d93f0b9d50dce3c0ee4245ef32db4c1427df6b36b4d1836d6d8bc6fea09674b470f020dae31e854be91129545449b56d4f0b48fd8e428f7a776826a684023c2aa70edef461e29a4f48e62b37b0b2bf1c39211ccc06cb4a337a0c922ff8b49ae0b20e6cc3abff4782959e759eed470465c43584f7220308dd677761800c3cee061e1ca790a0a03b44f3f64'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 461423 bytes downloaded
Downloaded and uncompressed: melbourne-housing-snapshot
Data source import complete.


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, i, filenames in os.walk('/kaggle/input'):
    for j in filenames:
        print(os.path.join(dirname,j))


/kaggle/input/melbourne-housing-snapshot/melb_data.csv


In [3]:
data=pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

In [4]:
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [5]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [6]:
xtrain=data.drop('Price', axis=1)

In [7]:
xtrain

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,S,Barry,26/08/2017,16.7,3150.0,4.0,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,SP,Williams,26/08/2017,6.8,3016.0,3.0,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,S,Raine,26/08/2017,6.8,3016.0,3.0,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,PI,Sweeney,26/08/2017,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [8]:
ytrain=data['Price']

In [9]:
ytrain

0        1480000.0
1        1035000.0
2        1465000.0
3         850000.0
4        1600000.0
           ...    
13575    1245000.0
13576    1031000.0
13577    1170000.0
13578    2500000.0
13579    1285000.0
Name: Price, Length: 13580, dtype: float64

In [10]:
xtrain.shape

(13580, 20)

In [11]:
ytrain.shape

(13580,)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
xtrain, xval, ytrain, yval=train_test_split(xtrain, ytrain, test_size=0.2, train_size=0.8, random_state=0)

In [14]:
xtrain.shape

(10864, 20)

In [15]:
xval.shape

(2716, 20)

In [16]:
ytrain.shape

(10864,)

In [17]:
yval.shape

(2716,)

In [18]:
for i in data.columns:
    print(i, data[i].dtype)

Suburb object
Address object
Rooms int64
Type object
Price float64
Method object
SellerG object
Date object
Distance float64
Postcode float64
Bedroom2 float64
Bathroom float64
Car float64
Landsize float64
BuildingArea float64
YearBuilt float64
CouncilArea object
Lattitude float64
Longtitude float64
Regionname object
Propertycount float64


In [19]:
xtrain=xtrain.select_dtypes(exclude=['object'])

In [20]:
xtrain.shape

(10864, 12)

In [21]:
xtrain

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.98670,13240.0
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.90050,6380.0
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.82200,3755.0
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.91580,8870.0
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,3,5.2,3056.0,3.0,1.0,2.0,212.0,NaN,NaN,-37.77695,144.95785,11918.0
3264,3,10.5,3081.0,3.0,1.0,1.0,748.0,101.0,1950.0,-37.74160,145.04810,2947.0
9845,4,6.7,3058.0,4.0,2.0,2.0,441.0,255.0,2002.0,-37.73572,144.97256,11204.0
10799,3,12.0,3073.0,3.0,1.0,1.0,606.0,NaN,NaN,-37.72057,145.02615,21650.0


In [22]:
xtrain['Car'].fillna(0)

12167    1.0
6524     1.0
8413     1.0
2919     1.0
6043     2.0
        ... 
13123    2.0
3264     1.0
9845     2.0
10799    1.0
2732     1.0
Name: Car, Length: 10864, dtype: float64

In [23]:
xtrain, xval, ytrain, yval=train_test_split(xtrain, ytrain, test_size=0.2, train_size=0.8, random_state=0)

In [24]:
xtrain.shape

(8691, 12)

In [25]:
xval.shape

(2173, 12)

In [26]:
ytrain.shape

(8691,)

In [27]:
yval.shape

(2173,)

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [29]:
xtrain.isnull().sum()

Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                42
Landsize            0
BuildingArea     4136
YearBuilt        3462
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

In [30]:
xtrain.drop(['BuildingArea','YearBuilt'], axis=1, inplace=True)
xval.drop(['BuildingArea','YearBuilt'], axis=1,inplace=True)
xval['Car']=xval['Car'].fillna(0)
xtrain['Car']=xtrain['Car'].fillna(0)

In [31]:
xval.isnull().sum()

Rooms            0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
Lattitude        0
Longtitude       0
Propertycount    0
dtype: int64

In [32]:
xtrain.isnull().sum()

Rooms            0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
Lattitude        0
Longtitude       0
Propertycount    0
dtype: int64

In [33]:
xtrain.fillna(0)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,Lattitude,Longtitude,Propertycount
12451,3,8.6,3019.0,3.0,1.0,4.0,685.0,-37.78481,144.86230,3589.0
9629,4,25.9,3754.0,4.0,2.0,2.0,512.0,-37.61133,145.09450,5812.0
13136,3,14.7,3151.0,3.0,1.0,1.0,189.0,-37.85777,145.14009,4048.0
9909,2,13.8,3188.0,2.0,1.0,1.0,348.0,-37.93557,145.00463,5454.0
8029,2,5.6,3101.0,2.0,1.0,1.0,0.0,-37.79770,145.03330,10331.0
...,...,...,...,...,...,...,...,...,...,...
993,4,13.1,3128.0,4.0,4.0,4.0,707.0,-37.81660,145.13540,4605.0
5883,2,6.1,3182.0,2.0,1.0,1.0,1357.0,-37.86010,144.98980,13240.0
4704,3,3.8,3207.0,3.0,1.0,0.0,171.0,-37.83850,144.94090,8648.0
1641,4,9.0,3126.0,4.0,2.0,1.0,653.0,-37.82920,145.08520,3265.0


In [34]:
def score_dataset(xtrain, xval, ytrain, yval):
    model=RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(xtrain,ytrain)
    prediction=model.predict(xval)
    return mean_absolute_error(yval, prediction)

In [35]:
score_dataset(xtrain, xval, ytrain, yval)

193211.1843753424

In [36]:
da=np.array([[2,4],[np.NaN,5],[8,np.NaN]])
da

array([[ 2.,  4.],
       [nan,  5.],
       [ 8., nan]])

In [37]:
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(da)

SimpleImputer()

In [38]:
imp.transform(da)

array([[2. , 4. ],
       [5. , 5. ],
       [8. , 4.5]])

In [39]:
daa=np.array([[29,np.NaN],[np.NaN,45],[85,77],[31,np.NaN],[np.NaN,20]])
imp.fit_transform(daa)

array([[29.        , 47.33333333],
       [48.33333333, 45.        ],
       [85.        , 77.        ],
       [31.        , 47.33333333],
       [48.33333333, 20.        ]])